# Lyrics Analysis Attempt (Genius)

In [49]:
import lyricsgenius as lg
import pandas as pd
import string 

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from urllib.request import Request, urlopen, quote
from bs4 import BeautifulSoup
import requests
from requests.exceptions import Timeout

In [47]:
token = "1VHE7q4FnW4XcMobQtv2RLtN-hc2ZgWMXm0zOTsIHptb9dOeNzddpuFRm4kT_CcZ"

In [48]:
## Creating Genius instance to scrap data
## Passed arguments to remove section headers, such as chorus
## as well as ignore non-songs and remixes, edits, etc. of songs
genius = lg.Genius(token, remove_section_headers=True,
                 skip_non_songs=True, excluded_terms=["Remix", "Live", "Edit", "Mix", "Club"])

In [5]:
sample_artists = ['The Beatles', 'The Rolling Stones']

In [6]:
for artist in sample_artists:
    artists = []
    titles = []
    albums = []
    years = []
    lyrics = []
    artist = genius.search_artist(artist, max_songs = 5, sort='popularity')
    songs = artist.songs
    song_number = 0
    #Append all information for each song in the previously created lists
    for song in songs:
        if song is not None:
            artists.append(song.artist)
            titles.append(song.title)
            albums.append(song.album)
            years.append(song.year[0:4])
            lyrics.append(song.lyrics)

Searching for songs by The Beatles...

Song 1: "Let It Be"
Song 2: "Yesterday"
Song 3: "Come Together"
Song 4: "Hey Jude"
Song 5: "Something"

Reached user-specified song limit (5).
Done. Found 5 songs.
Searching for songs by The Rolling Stones...

Song 1: "Sympathy for the Devil"
Song 2: "Paint It, Black"
Song 3: "Gimme Shelter"
Song 4: "You Can’t Always Get What You Want"
Song 5: "Wild Horses"

Reached user-specified song limit (5).
Done. Found 5 songs.


In [7]:
tracklist = pd.DataFrame({'artist':artists, 'title':titles, 'album':albums, 'year':years, 'lyrics':lyrics})

In [8]:
tracklist.head()

,artist,title,album,year,lyrics
0,The Rolling Stones,Sympathy for the Devil,Beggars Banquet,1969,"Yow....yow....yow\nHuh, huh, huh\n\nPlease all..."
1,The Rolling Stones,"Paint It, Black",Aftermath (US),1966,I see a red door and I want it painted black\n...
2,The Rolling Stones,Gimme Shelter,Let It Bleed,1969,"Oooooh\nOooooh\nOooooh\n\nOoh, a storm is thre..."
3,The Rolling Stones,You Can’t Always Get What You Want,Let It Bleed,1969,I saw her today at the reception\nA glass of w...
4,The Rolling Stones,Wild Horses,Sticky Fingers,1971,Childhood living is easy to do\nThe things you...


In [19]:
def getTop100Songs(URL):
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser")    
    song_items = html.find_all("div", class_="ye-chart-item__title")
    artist_items = html.find_all("div", class_="ye-chart-item__artist")
    comb = tuple(zip([item.get_text().strip() for item in song_items], [item.get_text().strip() for item in artist_items]))
    return comb

In [36]:
def getTop100Songs(URL):
    page = requests.get(URL)
    bs = BeautifulSoup(page.text, "lxml")
    rows = bs.find('table').find_all('tr')
    son_art = []
    for row in rows[1:]:
        cols = row.find_all(['td', 'th'])
        cols = [t.text.strip().strip('"') for t in cols]
        data = (cols[1], cols[2])
        son_art.append(data)
    return son_art

In [61]:
x = getTop100Songs("https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2006")

In [75]:
## Define function to scrape Billboard Top 100 Songs of the Year
## then pull lyric and song information from Genius.com
## utilizing lyricsgenius package, then convert to DataFrame
## I will start in 1970
def GetSongLyrics(start_year, end_year):
    
    ## Define Billboard Top 100 Scrapper
    def getTop100Songs(URL):
        page = requests.get(URL)
        bs = BeautifulSoup(page.text, "lxml")
        rows = bs.find('table').find_all('tr')
        son_art = []
        for row in rows[1:]:
            cols = row.find_all(['td', 'th'])
            cols = [t.text.strip().strip('"') for t in cols]
            data = (cols[1], cols[2])
            son_art.append(data)
        return son_art
    
    ## Define Genius API access token
    token = "1VHE7q4FnW4XcMobQtv2RLtN-hc2ZgWMXm0zOTsIHptb9dOeNzddpuFRm4kT_CcZ"

    ## Creating Genius instance to scrap data
    ## Passed arguments to remove section headers, such as chorus
    ## as well as ignore non-songs and remixes, edits, etc. of songs
    genius = lg.Genius(token, remove_section_headers=True,
                       skip_non_songs=True, excluded_terms=["Remix", "Live", "Edit", "Mix", "Club"])
    
    songs = []
    artists= []
    titles = []
    albums = []
    years = []
    lyrics = []
    
    for y in range(start_year, end_year+1):
        x = str("https://en.m.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_"+str(y))
        
        top100 = getTop100Songs(x)
        
        genius.timeout = 15
#        genius.sleep_time = 40
        
        for i in top100:
            retries = 0
            while retries < 3:
                try:
                    q = genius.search_song(i[0], i[1])
                except Timeout as e:
                    retries += 1
                    continue
                if q is not None:
                    songs.append(q)
                    years.append(y)
                break
    
    for song in songs:
        if song is not None:
            artists.append(song.artist)
            titles.append(song.title)
            albums.append(song.album)
            lyrics.append(song.lyrics)
        
    tracklist = pd.DataFrame({'artist':artists, 'title':titles, 'album':albums, 'year':years, 'lyrics':lyrics})
        
    return tracklist

In [76]:
df = GetSongLyrics(1970, 2020)
df.to_csv('top100_corpus.csv', encoding = 'utf-8', index=False)

Searching for "Bridge Over Troubled Water" by Simon & Garfunkel...
Done.
Searching for "(They Long to Be) Close to You" by The Carpenters...
Done.
Searching for "American Woman" by The Guess Who...
Done.
Searching for "Raindrops Keep Fallin' on My Head" by B.J. Thomas...
Done.
Searching for "War" by Edwin Starr...
Done.
Searching for "Ain't No Mountain High Enough" by Diana Ross...
Done.
Searching for "I'll Be There" by The Jackson 5...
Done.
Searching for "Get Ready" by Rare Earth...
Done.
Searching for "Let It Be" by The Beatles...
Done.
Searching for "Band of Gold" by Freda Payne...
Done.
Searching for "Mama Told Me (Not to Come)" by Three Dog Night...
Done.
Searching for "Everything Is Beautiful" by Ray Stevens...
Done.
Searching for "Make It with You" by Bread...
Done.
Searching for "Hitchin' a Ride" by Vanity Fare...
Done.
Searching for "ABC" by The Jackson 5...
Done.
Searching for "The Love You Save" by The Jackson 5...
Done.
Searching for "Cracklin' Rosie" by Neil Diamond...
Do

Done.
Searching for "Temptation Eyes" by The Grass Roots...
Done.
Searching for "Superstar" by The Carpenters...
Done.
Searching for "My Sweet Lord" by George Harrison...
Done.
Searching for "Sweet and Innocent" by Donny Osmond...
Done.
Searching for "Put Your Hand in the Hand" by Ocean...
Done.
Searching for "Chick-A-Boom (Don't Ya Jes' Love It)" by Daddy Dewdrop...
Done.
Searching for "For All We Know" by The Carpenters...
Done.
Searching for "Help Me Make It Through the Night" by Sammi Smith...
Done.
Searching for "Rainy Days and Mondays" by The Carpenters...
Done.
Searching for "If You Could Read My Mind" by Gordon Lightfoot...
Done.
Searching for "Gypsys, Tramps & Thieves" by Cher...
Done.
Searching for "Never Can Say Goodbye" by The Jackson 5...
Done.
Searching for "Rose Garden" by Lynn Anderson...
Done.
Searching for "Don't Pull Your Love" by Hamilton, Joe Frank & Reynolds...
Done.
Searching for "It Don't Come Easy" by Ringo Starr...
Done.
Searching for "Mr. Bojangles" by Nitty 

Done.
Searching for "Mother and Child Reunion" by Paul Simon...
Done.
Searching for "Where Is the Love" by Roberta Flack & Donny Hathaway...
Done.
Searching for "I'm Still in Love with You" by Al Green...
Done.
Searching for "Layla" by Derek and the Dominos...
Done.
Searching for "Day Dreaming" by Aretha Franklin...
Done.
Searching for "The Way of Love" by Cher...
Done.
Searching for "Black and White" by Three Dog Night...
Done.
Searching for "Sylvia's Mother" by Dr. Hook & The Medicine Show...
Done.
Searching for "Hurting Each Other" by The Carpenters...
Done.
Searching for "Coconut" by Harry Nilsson...
Done.
Searching for "Puppy Love" by Donny Osmond...
Done.
Searching for "You Don't Mess Around with Jim" by Jim Croce...
Done.
Searching for "Hot Rod Lincoln" by Commander Cody and His Lost Planet Airmen...
Done.
Searching for "A Cowboy's Work Is Never Done" by Sonny & Cher...
Done.
Searching for "Joy" by Apollo 100...
Specified song does not have a valid URL with lyrics.Rejecting.
Sea

Done.
Searching for "One of a Kind (Love Affair)" by The Spinners...
Done.
Searching for "Funny Face" by Donna Fargo...
Done.
Searching for "Funky Worm" by Ohio Players...
Done.
Searching for "Angie" by The Rolling Stones...
Done.
Searching for "Jambalaya (On the Bayou)" by Blue Ridge Rangers...
No results found for: 'Jambalaya (On the Bayou) Blue Ridge Rangers'
Searching for "Don't Expect Me to Be Your Friend" by Lobo...
Done.
Searching for "Break Up to Make Up" by The Stylistics...
Done.
Searching for "Daisy a Day" by Jud Strunk...
Done.
Searching for "Also Sprach Zarathustra (2001)" by Deodato...
Done.
Searching for "Stir It Up" by Johnny Nash...
Done.
Searching for "Money" by Pink Floyd...
Done.
Searching for "Gypsy Man" by War...
Done.
Searching for "The World Is a Ghetto" by War...
Done.
Searching for "Yes We Can Can" by The Pointer Sisters...
Done.
Searching for "Free Ride" by The Edgar Winter Group...
Done.
Searching for "Space Oddity" by David Bowie...
Done.
Searching for "It 

Done.
Searching for "Jive Talkin'" by Bee Gees...
Done.
Searching for "Best of My Love" by Eagles...
Done.
Searching for "Lovin' You" by Minnie Riperton...
Done.
Searching for "Kung Fu Fighting" by Carl Douglas...
Done.
Searching for "Black Water" by The Doobie Brothers...
Done.
Searching for "The Ballroom Blitz" by Sweet...
Done.
Searching for "(Hey Won't You Play) Another Somebody Done Somebody Wrong Song" by B.J. Thomas...
Done.
Searching for "He Don't Love You (Like I Love You)" by Tony Orlando and Dawn...
Done.
Searching for "At Seventeen" by Janis Ian...
Done.
Searching for "Pick Up the Pieces" by Average White Band...
Done.
Searching for "The Hustle" by Van McCoy & the Soul City Symphony...
Done.
Searching for "Lady Marmalade" by Labelle...
Done.
Searching for "Why Can't We Be Friends?" by War...
Done.
Searching for "Love Won't Let Me Wait" by Major Harris...
Done.
Searching for "Boogie On Reggae Woman" by Stevie Wonder...
Done.
Searching for "Angie Baby" by Helen Reddy...
Done.

...

Done.
Searching for "Love Never Felt So Good" by Michael Jackson and Justin Timberlake...
Done.
Searching for "Drunk on a Plane" by Dierks Bentley...
Done.
Searching for "Birthday" by Katy Perry...
Done.
Searching for "Bartender" by Lady Antebellum...
Done.
Searching for "La La La" by Naughty Boy featuring Sam Smith...
Done.
Searching for "Blurred Lines" by Robin Thicke featuring T.I. and Pharrell Williams...
Done.
Searching for "Do What U Want" by Lady Gaga featuring R. Kelly...
Done.
Searching for "Can't Remember to Forget You" by Shakira featuring Rihanna...
Done.
Searching for "Amnesia" by 5 Seconds of Summer...
Done.
Searching for "No Mediocre" by T.I. featuring Iggy Azalea...
Done.
Searching for "Come with Me Now" by KONGOS...
Done.
Searching for "Believe Me" by Lil Wayne featuring Drake...
Done.
Searching for "23" by Mike WiLL Made-It featuring Miley Cyrus, Wiz Khalifa, and Juicy J...
Done.
Searching for "Beachin'" by Jake Owen...
Done.
Searching for "White Walls" by Macklemore 

Done.
Searching for "Closer" by The Chainsmokers featuring Halsey...
Done.
Searching for "Cheap Thrills" by Sia featuring Sean Paul...
Specified song does not contain lyrics. Rejecting.
Searching for "7 Years" by Lukas Graham...
Done.
Searching for "Needed Me" by Rihanna...
Done.
Searching for "My House" by Flo Rida...
Done.
Searching for "I Took a Pill in Ibiza" by Mike Posner...
Done.
Searching for "Work from Home" by Fifth Harmony featuring Ty Dolla $ign...
Done.
Searching for "This Is What You Came For" by Calvin Harris featuring Rihanna...
Done.
Searching for "Cake by the Ocean" by DNCE...
Done.
Searching for "Me, Myself & I" by G-Eazy and Bebe Rexha...
Done.
Searching for "Ride" by Twenty One Pilots...
Done.
Searching for "Heathens" by Twenty One Pilots...
Done.
Searching for "Pillowtalk" by Zayn...
Done.
Searching for "Stitches" by Shawn Mendes...
Done.
Searching for "Hotline Bling" by Drake...
Done.
Searching for "Cold Water" by Major Lazer featuring Justin Bieber and MØ...
Don

Done.
Searching for "Look What You Made Me Do" by Taylor Swift...
Done.
Searching for "Castle on the Hill" by Ed Sheeran...
Done.
Searching for "Bad Things" by Machine Gun Kelly and Camila Cabello...
Done.
Searching for "Paris" by The Chainsmokers...
Done.
Searching for "Side to Side" by Ariana Grande featuring Nicki Minaj...
Done.
Searching for "Rockabye" by Clean Bandit featuring Sean Paul and Anne-Marie...
Done.
Searching for "Feel It Still" by Portugal. The Man...
Done.
Searching for "Let Me Love You" by DJ Snake featuring Justin Bieber...
Done.
Searching for "Sorry Not Sorry" by Demi Lovato...
Done.
Searching for "Bank Account" by 21 Savage...
Done.
Searching for "Can't Stop the Feeling!" by Justin Timberlake...
Done.
Searching for "Mi Gente" by J Balvin and Willy William featuring Beyoncé...
Specified song does not contain lyrics. Rejecting.
Searching for "Thunder" by Imagine Dragons...
Done.
Searching for "T-Shirt" by Migos...
Done.
Searching for "Rake It Up" by Yo Gotti featuri

Done.
Searching for "No Brainer" by DJ Khaled featuring Justin Bieber, Chance the Rapper and Quavo...
Specified song does not have a valid URL with lyrics.Rejecting.
Searching for "Plain Jane" by ASAP Ferg featuring Nicki Minaj...
Done.
Searching for "Sky Walker" by Miguel featuring Travis Scott...
Done.
Searching for "Marry Me" by Thomas Rhett...
Done.
Searching for "Eastside" by Benny Blanco, Halsey and Khalid...
Done.
Searching for "Call Out My Name" by The Weeknd...
Done.
Searching for "King's Dead" by Jay Rock, Kendrick Lamar, Future and James Blake...
Done.
Searching for "Happier" by Marshmello and Bastille...
Done.
Searching for "Te Boté" by Nio García, Darell and Casper Mágico featuring Bad Bunny, Nicky Jam and Ozuna...
Done.
Searching for "Simple" by Florida Georgia Line...
Done.
Searching for "Lemon" by N.E.R.D and Rihanna...
Done.
Searching for "1-800-273-8255" by Logic featuring Alessia Cara and Khalid...
Done.
Searching for "Say Something" by Justin Timberlake featuring Ch

Done.
Searching for "Rockstar" by DaBaby featuring Roddy Ricch...
Specified song does not contain lyrics. Rejecting.
Searching for "Adore You" by Harry Styles...
Done.
Searching for "Life Is Good" by Future featuring Drake...
Done.
Searching for "Memories" by Maroon 5...
Done.
Searching for "The Bones" by Maren Morris...
Done.
Searching for "Someone You Loved" by Lewis Capaldi...
Done.
Searching for "Say So" by Doja Cat...
Done.
Searching for "I Hope" by Gabby Barrett featuring Charlie Puth...
Specified song does not contain lyrics. Rejecting.
Searching for "Whats Poppin" by Jack Harlow featuring DaBaby, Tory Lanez and Lil Wayne...
Specified song does not contain lyrics. Rejecting.
Searching for "Dance Monkey" by Tones and I...
Done.
Searching for "Savage" by Megan Thee Stallion featuring Beyoncé...
Specified song does not contain lyrics. Rejecting.
Searching for "Roxanne" by Arizona Zervas...
Done.
Searching for "Intentions" by Justin Bieber featuring Quavo...
Done.
Searching for "Eve

In [124]:
OHCO = ['decade', 'year', 'artist', 'album', 'title', 'verse', 'line', 'token']

In [120]:
## LyricsGenius library isn't perfect and for some reason
## for certain artists, it pulled other items, like chapters
## from Melville or Joyce novels, or the script of the film
## Boogie Nights. Went in & manually cleaned; were fairly easy
## to identify, as they would be long lines of data taking up
## multiple rows, which did not follow the standard format of
## all the actual song lyrics in the csv file

lyr = pd.read_csv("top100_corpus_cleaned.csv")

## Adding a column to identify song decade
decades = []

for year in lyr['year']:
    if 1970 <= year < 1980:
        decades.append("70s")
    if 1980 <= year < 1990:
        decades.append("80s")
    if 1990 <= year < 2000:
        decades.append("90s")
    if 2000 <= year < 2010:
        decades.append("00s")
    if 2010 <= year :
        decades.append("10s")
lyr['decade'] = decades

lyr = lyr.set_index(OHCO[:5])
lyr.head()

lyrics
decade year artist            album                             title                                                                               
70s    1970 Simon & Garfunkel Bridge Over Troubled Water        Bridge Over Troubled Water         When you're weary\nFeeling small\nWhen tears a...
            ​The Guess Who    American Woman                    American Woman                     American woman gonna mess your mind\nAmerican ...
            B.J. Thomas       Raindrops Keep Fallin’ on My Head Raindrops Keep Fallin’ on My Head  Raindrops keep fallin' on my head\nAnd just li...
            Edwin Starr       War & Peace                       War                                (War, huh) Yeah!\n(What is it good for?) Absol...
            Diana Ross        Diana Ross (1970)                 Ain’t No Mountain High Enough      If you need me, call me\nNo matter where you a...

In [122]:
lyri = lyr['lyrics'].str.split(r'\n\n+', expand=True).stack()\
    .to_frame().rename(columns={0:'verse_str'})

lyri.index.names = OHCO[:6]

In [130]:
lyri2 = lyri['verse_str'].str.split(r'\n', expand=True).stack()\
    .to_frame().rename(columns={0:'line_str'})

lyri2.index.names = OHCO[:7]

In [146]:
lyri2.head()

line_str
decade year artist            album                      title                      verse line                             
70s    1970 Simon & Garfunkel Bridge Over Troubled Water Bridge Over Troubled Water 0     0               When you're weary
                                                                                          1                   Feeling small
                                                                                          2     When tears are in your eyes
                                                                                          3             I will dry them all
                                                                                          4                I'm on your side

In [154]:
lyri2.iloc[170][0]

'If you need me, call me'